# HW 8: Integer programs and duality

## 1. ABC investments

!!!!!!bad codes!

The variables are  
1. $S[i]$, (S is the short for selection), which only has 0 and 1 inside, indicating if a investment is made for the $i_{th}$ option;  
2. $I[i]$, (I is the short for investment), indicating how much investment is made for the $i_{th}$ option;  
  
So we can see the investment is actually expressed by $S^\top I$  
The constraints are:  
1. $\sum_i S_iI_i = 80$;  
2. $S$ only contains 0 and 1;  
3. $S_5I_5 <= S_2I_2 + S_4I_4 + S_6I_6$;  
4. if $S_3 = 1$, then $S_6 = 1$;  
5. $S_iMin_i \le  S_iI_i \le S_iMax_i$ where $Min_i$ and $Max_i$ is the minimum and maximum investment for the $i_{th}$ option;  
  
The obejective is to maximum $\sum_i S_iI_iP_i$, where $P_i$ is the return of the $i_{th}$ option.

In [1]:
#using JuMP, Gurobi, PyPlot

In [2]:
#=
option = [3 27 13; 2 12 9; 9 35 17; 5 15 10; 12 46 22; 4 18 12];
m1 = Model(Gurobi.Optimizer);
set_silent(m1);

select1 = @variable(m1, [1:6,1:1], Bin); ## option3 == 0
select2 = @variable(m1, [1:6,1:1], Bin); ## option3 and option6 == 1

@constraint(m1, select1[3] == 0);
@constraint(m1, select2[3] == 1);
@constraint(m1, select2[6] == 1);

#invest1 = @variable(m1, [1:6,1:1]);
@variable(m1, invest1[1:6]);

@constraint(m1, sum(select1 .*invest1) == 80);
@constraint(m1, sum(select2 .*invest1) == 80);

@constraint(m1, select1[5] * invest1[5] <= (select1[2] *invest1[2] + select1[4] *invest1[4] + select1[6] *invest1[6]));
@constraint(m1, select2[5] * invest1[5] <= (select2[2] *invest1[2] + select2[4] *invest1[4] + select2[6] *invest1[6]));
for i = 1:6
    @constraint(m1, select1[i]*invest1[i] >=  select1[i]*option[i,1]);
    @constraint(m1, select2[i]*invest1[i] >=  select2[i]*option[i,1]);
    @constraint(m1, select1[i]*invest1[i] <=  select1[i]*option[i,2]);
    @constraint(m1, select2[i]*invest1[i] <=  select2[i]*option[i,2]);
end

@variable(m1, Profit);
@constraint(m1, Profit <= sum(select1 .*invest1 .*option[:, 3] ./100) );
@constraint(m1, Profit <= sum(select2 .*invest1 .*option[:, 3] ./100) );

@objective(m1, Max, Profit);

optimize!(m1);
=#

In [3]:
#=
println("The maximum return is \$$(round.(objective_value(m1), digits =6)) million ");
println("The investment is $(round.(value.(invest1), digits =6)) (million dollars) for these six options.");
=#

The variables are  
1. $S[i]$, (S is the short for selection), which only has 0 and 1 inside, indicating if a investment is made for the $i_{th}$ option;  
2. $I[i]$, (I is the short for investment), indicating how much investment is made for the $i_{th}$ option;  
  
So we can see the investment is actually expressed by $S^\top I$  
The constraints are:  
1. $\sum_i I_i = 80$;  
2. $S$ only contains 0 and 1;  
3. $I_5 <= I_2 + I_4 + I_6$ ;  
4. if $S_3 = 1$, then $S_6 = 1$, or, if $S_6 = 0$, then $S_3 = 0$;  
5. $S_iMin_i \le  I_i \le S_iMax_i$ where $Min_i$ and $Max_i$ is the minimum and maximum investment for the $i_{th}$ option;  
  
The obejective is to maximum $\sum_i I_iP_i$, where $P_i$ is the return of the $i_{th}$ option.

In [4]:
m1_test = Model(Gurobi.Optimizer);
set_silent(m1_test);
option = [3 27 13; 2 12 9; 9 35 17; 5 15 10; 12 46 22; 4 18 12];

select = @variable(m1_test, [1:6, 1:1], Bin);
invest = @variable(m1_test, [1:6,1:1]);

@constraint(m1_test, sum(invest) == 80);
@constraint(m1_test, invest[5]<=(invest[2] + invest[4] + invest[6]));

for i=1:6
    if i==3
        @constraint(m1_test, invest[i] <= select[i]*select[6]*option[i,2]); ## if opt6 is 0, then invest3 is 0
        @constraint(m1_test, invest[i] >= select[i]*select[6]*option[i,1]); ## if opt6 is 1, then invest3 is controlled by select3
    else
        @constraint(m1_test, invest[i] <= select[i]*option[i,2]);
        @constraint(m1_test, invest[i] >= select[i]*option[i,1]);
    end            
end

@objective(m1_test, Max, sum(select .*invest .*option[:, 3] ./100));
optimize!(m1_test);


UndefVarError: UndefVarError: Gurobi not defined

In [5]:
println("The maximum return is \$$(round.(objective_value(m1_test), digits =6)) million ");
println("The investment is $(round.(value.(invest), digits =6)) (million dollars) for these six options.");

UndefVarError: UndefVarError: objective_value not defined

## 2. Lagrangian duality

### a)

The variables are $x_1$ and $x_2$;  
The constraint is $x_1 \ge 1$;  
The objective is to minimize $\frac{1}{2}(x_1^2 + x_2^2)$;

In [6]:
using Gurobi, JuMP;

In [7]:
m2a = Model(Gurobi.Optimizer);
set_silent(m2a);
x = @variable(m2a, [1:2, 1:1]);
@constraint(m2a, x[1] >= 1  );
@objective(m2a, Min, (x[1]^2 + x[2]^2)/2);
optimize!(m2a);
primal_optimal = objective_value(m2a);

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [8]:
println("The optimal primal value p* is $(primal_optimal)");

The optimal primal value p* is 0.5


### b)

The Lagrangian $L(x_1, x_2, \lambda) = \frac{1}{2}(x_1^2 + x_2^2) + \lambda(1-x_1)$  
$L(x_1, x_2, \lambda) = \frac{1}{2}(x_1^2 + x_2^2 - 2\lambda x_1) + \lambda = \frac{1}{2}(x_1^2 + x_2^2 - 2\lambda x_1 + \lambda^2) - \frac{1}{2}\lambda^2 + \lambda = \frac{1}{2}((x_1 - \lambda)^2 + x_2^2) - \frac{1}{2}\lambda^2 + \lambda$  
The dual $g(\lambda) = min\frac{1}{2}((x_1 - \lambda)^2 + x_2^2)  - \frac{1}{2}\lambda^2 + \lambda$  
As $min\frac{1}{2}((x_1 - \lambda)^2 + x_2^2) = 0$, we have  
$g(\lambda) = - \frac{1}{2}\lambda^2 + \lambda \quad (s.t \quad  \lambda \ge 0)$

### c)

In [9]:
m2b = Model(Gurobi.Optimizer);
set_silent(m2b);
@variable(m2b, lambda >= 0);
@objective(m2b, Max, -0.5 * lambda^2 + lambda );
optimize!(m2b);
dual_optimal = objective_value(m2b);

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [10]:
println("The optimal dual value d* is $(dual_optimal)");

The optimal dual value d* is 0.5


### d)

```Slater condition is staisfied in this question. Thus the strong duality holds.```